In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn import metrics
import xgboost as xgb

In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df.shape

(71236, 50)

In [4]:
# df = df.drop_duplicates(subset= ['patient_id'], keep = 'first')
# df.shape

In [5]:
# df['patient_id'] = df['patient_id'].astype(str)

In [6]:
df['race'].unique()

array(['Caucasian', 'AfricanAmerican', 'Hispanic', 'Asian', 'Other', nan],
      dtype=object)

In [7]:
df.isnull().sum()

enc_id                          0
patient_id                      0
race                         1621
gender                          0
age                             0
weight                      68986
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  28178
medical_specialty           34930
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         15
diag_2                        244
diag_3                        989
number_diagnoses                0
max_glu_serum               67515
A1Cresult                   59356
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [8]:
df['diag_1'].value_counts()

diag_1
428    4836
414    4618
786    2791
410    2537
486    2461
       ... 
352       1
133       1
10        1
833       1
906       1
Name: count, Length: 684, dtype: int64

## Outlier Removal

In [9]:
for i in range(1,29):
    filtered_df = df[df['admission_type_id'] == i]

# Calculate the ratio of counts for 0s, 1s, and 2s in readmission_id
    count_0 = (filtered_df['readmission_id'] == 0).sum()
    count_1 = (filtered_df['readmission_id'] == 1).sum()
    count_2 = (filtered_df['readmission_id'] == 2).sum()

    total_counts = count_0 + count_1 + count_2
    if(total_counts==0):
        print("Remove:",i)
        continue
# Calculate the ratios
    ratio_0 = count_0 / total_counts
    ratio_1 = count_1 / total_counts
    ratio_2 = count_2 / total_counts
    print("ID=",i)
# Display the ratios
    print(f'Ratio of 0s: {ratio_0:.2f}')
    print(f'Ratio of 1s: {ratio_1:.2f}')
    print(f'Ratio of 2s: {ratio_2:.2f}')

ID= 1
Ratio of 0s: 0.11
Ratio of 1s: 0.36
Ratio of 2s: 0.53
ID= 2
Ratio of 0s: 0.11
Ratio of 1s: 0.35
Ratio of 2s: 0.54
ID= 3
Ratio of 0s: 0.10
Ratio of 1s: 0.30
Ratio of 2s: 0.59
ID= 4
Ratio of 0s: 0.12
Ratio of 1s: 0.12
Ratio of 2s: 0.75
ID= 5
Ratio of 0s: 0.10
Ratio of 1s: 0.36
Ratio of 2s: 0.54
ID= 6
Ratio of 0s: 0.11
Ratio of 1s: 0.43
Ratio of 2s: 0.46
ID= 7
Ratio of 0s: 0.00
Ratio of 1s: 0.00
Ratio of 2s: 1.00
ID= 8
Ratio of 0s: 0.08
Ratio of 1s: 0.26
Ratio of 2s: 0.67
Remove: 9
Remove: 10
Remove: 11
Remove: 12
Remove: 13
Remove: 14
Remove: 15
Remove: 16
Remove: 17
Remove: 18
Remove: 19
Remove: 20
Remove: 21
Remove: 22
Remove: 23
Remove: 24
Remove: 25
Remove: 26
Remove: 27
Remove: 28


In [10]:
print(df['admission_type_id'].value_counts())

admission_type_id
1    37831
3    13188
2    12979
6     3675
5     3321
8      227
4        8
7        7
Name: count, dtype: int64


In [11]:
df['admission_type_id'] = df['admission_type_id'].replace(2,1)
df['admission_type_id'] = df['admission_type_id'].replace(7,4)
df['admission_type_id'] = df['admission_type_id'].replace(5,0)
df['admission_type_id'] = df['admission_type_id'].replace(3,0)
df['admission_type_id'] = df['admission_type_id'].replace(6,0)
df['admission_type_id'] = df['admission_type_id'].replace(8,0)
print(df['admission_type_id'].value_counts())

admission_type_id
1    50810
0    20411
4       15
Name: count, dtype: int64


In [12]:
print(df['discharge_disposition_id'].value_counts())

discharge_disposition_id
1     42161
3      9786
6      8984
18     2603
2      1472
22     1385
11     1154
5       847
25      716
4       570
7       444
23      282
13      271
14      269
28      101
8        78
15       37
24       33
9        15
16        8
17        7
19        5
10        3
27        2
12        2
20        1
Name: count, dtype: int64


In [13]:
for i in range(1,30):
    filtered_df = df[df['discharge_disposition_id'] == i]
# Calculate the ratio of counts for 0s, 1s, and 2s in readmission_id
    count_0 = (filtered_df['readmission_id'] == 0).sum()
    count_1 = (filtered_df['readmission_id'] == 1).sum()
    count_2 = (filtered_df['readmission_id'] == 2).sum()

    total_counts = count_0 + count_1 + count_2
    if(total_counts==0):
        print("Remove:",i)
        continue
# Calculate the ratios
    ratio_0 = count_0 / total_counts
    ratio_1 = count_1 / total_counts
    ratio_2 = count_2 / total_counts
    print("ID=",i)
# Display the ratios
    print(f'Ratio of 0s: {ratio_0:.2f}')
    print(f'Ratio of 1s: {ratio_1:.2f}')
    print(f'Ratio of 2s: {ratio_2:.2f}')

ID= 1
Ratio of 0s: 0.09
Ratio of 1s: 0.36
Ratio of 2s: 0.55
ID= 2
Ratio of 0s: 0.16
Ratio of 1s: 0.31
Ratio of 2s: 0.53
ID= 3
Ratio of 0s: 0.14
Ratio of 1s: 0.36
Ratio of 2s: 0.50
ID= 4
Ratio of 0s: 0.14
Ratio of 1s: 0.35
Ratio of 2s: 0.52
ID= 5
Ratio of 0s: 0.22
Ratio of 1s: 0.29
Ratio of 2s: 0.49
ID= 6
Ratio of 0s: 0.13
Ratio of 1s: 0.42
Ratio of 2s: 0.46
ID= 7
Ratio of 0s: 0.14
Ratio of 1s: 0.35
Ratio of 2s: 0.50
ID= 8
Ratio of 0s: 0.15
Ratio of 1s: 0.33
Ratio of 2s: 0.51
ID= 9
Ratio of 0s: 0.40
Ratio of 1s: 0.07
Ratio of 2s: 0.53
ID= 10
Ratio of 0s: 0.00
Ratio of 1s: 0.67
Ratio of 2s: 0.33
ID= 11
Ratio of 0s: 0.00
Ratio of 1s: 0.00
Ratio of 2s: 1.00
ID= 12
Ratio of 0s: 0.50
Ratio of 1s: 0.00
Ratio of 2s: 0.50
ID= 13
Ratio of 0s: 0.04
Ratio of 1s: 0.08
Ratio of 2s: 0.88
ID= 14
Ratio of 0s: 0.07
Ratio of 1s: 0.02
Ratio of 2s: 0.91
ID= 15
Ratio of 0s: 0.41
Ratio of 1s: 0.30
Ratio of 2s: 0.30
ID= 16
Ratio of 0s: 0.00
Ratio of 1s: 0.38
Ratio of 2s: 0.62
ID= 17
Ratio of 0s: 0.00
Ratio of

In [14]:
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(6,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(8,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(9,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(13,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(3,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(4,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(5,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(14,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(22,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(23,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(24,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(12,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(15,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(16,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(17,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(25,18)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(26,18)
print(df['discharge_disposition_id'].value_counts())

discharge_disposition_id
1     51509
2     14644
18     3319
11     1154
7       444
28      101
10       57
19        5
27        2
20        1
Name: count, dtype: int64


In [15]:
print(df['admission_source_id'].value_counts())

admission_source_id
7     40268
1     20701
17     4683
4      2239
6      1602
2       775
5       613
3       134
20      115
9        79
8        11
22        7
10        3
14        2
11        2
25        1
13        1
Name: count, dtype: int64


In [16]:
for i in range(1,30):
    filtered_df = df[df['admission_source_id'] == i]
# Calculate the ratio of counts for 0s, 1s, and 2s in readmission_id
    count_0 = (filtered_df['readmission_id'] == 0).sum()
    count_1 = (filtered_df['readmission_id'] == 1).sum()
    count_2 = (filtered_df['readmission_id'] == 2).sum()

    total_counts = count_0 + count_1 + count_2
    if(total_counts==0):
        print("Remove:",i)
        continue
# Calculate the ratios
    ratio_0 = count_0 / total_counts
    ratio_1 = count_1 / total_counts
    ratio_2 = count_2 / total_counts
    print("ID=",i)
# Display the ratios
    print(f'Ratio of 0s: {ratio_0:.2f}')
    print(f'Ratio of 1s: {ratio_1:.2f}')
    print(f'Ratio of 2s: {ratio_2:.2f}')

ID= 1
Ratio of 0s: 0.11
Ratio of 1s: 0.33
Ratio of 2s: 0.57
ID= 2
Ratio of 0s: 0.11
Ratio of 1s: 0.28
Ratio of 2s: 0.61
ID= 3
Ratio of 0s: 0.14
Ratio of 1s: 0.35
Ratio of 2s: 0.51
ID= 4
Ratio of 0s: 0.09
Ratio of 1s: 0.20
Ratio of 2s: 0.71
ID= 5
Ratio of 0s: 0.12
Ratio of 1s: 0.28
Ratio of 2s: 0.61
ID= 6
Ratio of 0s: 0.10
Ratio of 1s: 0.17
Ratio of 2s: 0.73
ID= 7
Ratio of 0s: 0.12
Ratio of 1s: 0.38
Ratio of 2s: 0.51
ID= 8
Ratio of 0s: 0.18
Ratio of 1s: 0.27
Ratio of 2s: 0.55
ID= 9
Ratio of 0s: 0.09
Ratio of 1s: 0.14
Ratio of 2s: 0.77
ID= 10
Ratio of 0s: 0.00
Ratio of 1s: 0.00
Ratio of 2s: 1.00
ID= 11
Ratio of 0s: 0.00
Ratio of 1s: 0.00
Ratio of 2s: 1.00
Remove: 12
ID= 13
Ratio of 0s: 0.00
Ratio of 1s: 0.00
Ratio of 2s: 1.00
ID= 14
Ratio of 0s: 0.00
Ratio of 1s: 0.00
Ratio of 2s: 1.00
Remove: 15
Remove: 16
ID= 17
Ratio of 0s: 0.10
Ratio of 1s: 0.37
Ratio of 2s: 0.53
Remove: 18
Remove: 19
ID= 20
Ratio of 0s: 0.14
Ratio of 1s: 0.52
Ratio of 2s: 0.34
Remove: 21
ID= 22
Ratio of 0s: 0.29
Rat

In [17]:
df['admission_source_id'] = df['admission_source_id'].replace(2,1)
df['admission_source_id'] = df['admission_source_id'].replace(3,1)
df['admission_source_id'] = df['admission_source_id'].replace(5,4)
df['admission_source_id'] = df['admission_source_id'].replace(6,4)
df['admission_source_id'] = df['admission_source_id'].replace(10,4)
df['admission_source_id'] = df['admission_source_id'].replace(22,4)
df['admission_source_id'] = df['admission_source_id'].replace(25,4)
df['admission_source_id'] = df['admission_source_id'].replace(15,9)
df['admission_source_id'] = df['admission_source_id'].replace(17,9)
df['admission_source_id'] = df['admission_source_id'].replace(20,9)
df['admission_source_id'] = df['admission_source_id'].replace(21,9)
df['admission_source_id'] = df['admission_source_id'].replace(13,11)
df['admission_source_id'] = df['admission_source_id'].replace(14,11)
print(df['admission_source_id'].value_counts())

admission_source_id
7     40268
1     21610
9      4877
4      4465
8        11
11        5
Name: count, dtype: int64


In [18]:
def plotOutliers(df, columns):
    for col in columns:
        plt.figure(figsize=(8, 6))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col}')
        plt.show()

def getCountOutliers(df):
    for col in df.columns:
        if col not in ['num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses']:
            continue
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (1.5 * iqr)
        upper_bound = q3 + (1.5 * iqr)
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
        print(f"Col Name: {col} | Col Total Values: {len(df[col])} | Outliers: {outliers}")

In [19]:
getCountOutliers(df)

Col Name: num_lab_procedures | Col Total Values: 71236 | Outliers: 103
Col Name: num_procedures | Col Total Values: 71236 | Outliers: 3461
Col Name: num_medications | Col Total Values: 71236 | Outliers: 1801
Col Name: number_outpatient | Col Total Values: 71236 | Outliers: 11698
Col Name: number_emergency | Col Total Values: 71236 | Outliers: 7962
Col Name: number_inpatient | Col Total Values: 71236 | Outliers: 4901
Col Name: number_diagnoses | Col Total Values: 71236 | Outliers: 185


In [20]:
def remove_outliers(df, columns):
    new_df = df.copy()
    for col in columns:
        Q1 = new_df[col].quantile(0.25)
        Q3 = new_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # new_df = new_df[(new_df[col] >= lower_bound) & (new_df[col] <= upper_bound)]
        # instead of removing outliers, we will cap them
        new_df.loc[new_df[col] < lower_bound, col] = lower_bound
        new_df.loc[new_df[col] > upper_bound, col] = upper_bound
    return new_df

In [21]:
# print(f"Shape before removing outliers: {df.shape}")
# df = remove_outliers(df, ['num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses'])
# print(f"Shape after removing outliers: {df.shape}")

## Pre-Processing

In [22]:
df['age'].value_counts()

age
[70-80)     18179
[60-70)     15801
[50-60)     12080
[80-90)     12037
[40-50)      6785
[30-40)      2650
[90-100)     1940
[20-30)      1165
[10-20)       495
[0-10)        104
Name: count, dtype: int64

In [23]:
# for i in range(0,10):
#     df['age'] = df['age'].replace('['+str(10*i)+'-'+str(10*(i+1))+')',i)
# df['age'].value_counts()

In [24]:
# replace A1Cresult of df when NaN with NONE string
df['A1Cresult'] = df['A1Cresult'].replace(np.nan, 'None')
df['max_glu_serum'] = df['max_glu_serum'].replace(np.nan, 'None')

#### The columns citoglipton and examide have just 1 value, thus they will not affect the classification

In [25]:
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)
df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)
df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)
# categorical_df=df.drop(['enc_id','weight','race','medical_specialty',
#        'time_in_hospital', 'payer_code',
#        'num_lab_procedures', 'num_procedures', 'num_medications',
#        'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses','A1Cresult','max_glu_serum'],axis=1)
categorical_df=df.drop(['enc_id','weight','race','medical_specialty',
       'time_in_hospital', 'payer_code',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses'],axis=1)
print(categorical_df.shape)
print(categorical_df.columns)

(71236, 37)
Index(['patient_id', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'diag_1', 'diag_2',
       'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide',
       'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',
       'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
       'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
       'glipizide-metformin', 'glimepiride-pioglitazone',
       'metformin-rosiglitazone', 'metformin-pioglitazone', 'change',
       'diabetesMed', 'readmission_id'],
      dtype='object')


In [26]:
categorical_df.head(20)

,patient_id,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,diag_1,diag_2,diag_3,max_glu_serum,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,2488608,1,[60-70),1,2,4,410,428,414,None,...,No,Steady,No,No,No,No,No,1,1,2
1,52133202,1,[70-80),1,1,1,414,424,428,None,...,No,No,No,No,No,No,No,0,1,1
2,40945509,0,[70-80),0,2,1,518,514,997,None,...,No,Up,No,No,No,No,No,1,1,1
3,38850777,0,[50-60),1,1,7,592,591,250,None,...,No,No,No,No,No,No,No,0,1,2
4,72738225,0,[80-90),1,1,7,250.6,713,707,None,...,No,Down,No,No,No,No,No,1,1,2
5,56850768,0,[70-80),1,1,7,599,496,276,None,...,No,Steady,No,No,No,No,No,1,1,1
6,3570651,1,[80-90),1,2,7,531,428,584,None,...,No,No,No,No,No,No,No,0,1,1
7,3132000,1,[60-70),0,18,4,V57,707,250.7,None,...,No,Steady,No,No,No,No,No,0,1,1
8,58997583,1,[70-80),0,1,7,518,428,496,None,...,No,Steady,No,No,No,No,No,0,1,2
9,139051085,1,[60-70),1,1,7,850,250.01,805,None,...,No,Down,No,No,No,No,No,1,1,2


In [27]:
# categorical_df = categorical_df.dropna()

## Feature Engineering for patient details

In [28]:
numerical_df = df[['num_medications','time_in_hospital','num_procedures','num_lab_procedures','number_diagnoses','number_emergency','number_inpatient',
'number_outpatient']]

In [29]:
drugs = ['metformin', 'repaglinide','glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone','insulin','citoglipton','nateglinide','chlorpropamide','acarbose','miglitol','glyburide-metformin', 'tolazamide','metformin-pioglitazone', 'metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin','troglitazone', 'tolbutamide', 'acetohexamide','examide']

In [30]:
print(len(drugs))

23


In [31]:
categorical_df['change'] = categorical_df['change'].replace('Ch', 1)
categorical_df['change'] = categorical_df['change'].replace('No', 0)
categorical_df['gender'] = categorical_df['gender'].replace('Male', 1)
categorical_df['gender'] = categorical_df['gender'].replace('Female', 0)
categorical_df['gender'] = categorical_df['gender'].replace('Unknown/Invalid', 0)
categorical_df['diabetesMed'] = categorical_df['diabetesMed'].replace('Yes', 1)
categorical_df['diabetesMed'] = categorical_df['diabetesMed'].replace('No', 0)
# categorical_df["drug_change_sum"]=0
# for col in drugs:
#     categorical_df[col] = categorical_df[col].replace('No', 0)
#     categorical_df[col] = categorical_df[col].replace('Steady', 0)
#     categorical_df[col] = categorical_df[col].replace('Up', 1)
#     categorical_df[col] = categorical_df[col].replace('Down', 1)
# for col in drugs:
#     categorical_df["drug_change_sum"]+=categorical_df[col]
# categorical_df=categorical_df.drop(columns=drugs,axis=1)
# print(categorical_df.shape)

In [32]:
# for i in range(0,4):
#     for col in drugs:
#         filtered_df = df[df[col] == i]
#         count_0 = (filtered_df['admission_source_id'] == 0).sum()
#         count_1 = (filtered_df['admission_source_id'] == 1).sum()
#         count_2 = (filtered_df['admission_source_id'] == 2).sum()
#         total_counts = count_0 + count_1 + count_2
#         if(total_counts==0):
#             print("Remove:",i)
#         continue
#         ratio_0 = count_0 / total_counts
#         ratio_1 = count_1 / total_counts
#         ratio_2 = count_2 / total_counts
#         print("ID=",i)
#     print(f'Ratio of 0s: {ratio_0:.2f}')
#     print(f'Ratio of 1s: {ratio_1:.2f}')
#     print(f'Ratio of 2s: {ratio_2:.2f}')

In [33]:
categorical_df.fillna(0)

,patient_id,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,diag_1,diag_2,diag_3,max_glu_serum,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,2488608,1,[60-70),1,2,4,410,428,414,None,...,No,Steady,No,No,No,No,No,1,1,2
1,52133202,1,[70-80),1,1,1,414,424,428,None,...,No,No,No,No,No,No,No,0,1,1
2,40945509,0,[70-80),0,2,1,518,514,997,None,...,No,Up,No,No,No,No,No,1,1,1
3,38850777,0,[50-60),1,1,7,592,591,250,None,...,No,No,No,No,No,No,No,0,1,2
4,72738225,0,[80-90),1,1,7,250.6,713,707,None,...,No,Down,No,No,No,No,No,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,85063725,1,[70-80),1,1,7,250.42,276,403,None,...,No,No,No,No,No,No,No,0,1,1
71232,86244345,1,[90-100),1,2,7,38,518,486,None,...,No,No,No,No,No,No,No,0,0,2
71233,5131368,1,[70-80),0,1,1,414,285,276,None,...,No,Steady,No,No,No,No,No,1,1,2
71234,85969035,1,[50-60),1,1,4,572,571,287,None,...,No,Steady,No,No,No,No,No,0,1,2


## Feature Engineering for Drug columns

In [34]:
# categorical_df[[drugs[0],drugs[1]]].head(10)
# there are 23 drugs, 4 categories each: No, Steady, Up, Down
# we will create 4 new columns for each row: number_of_no, number_of_steady, number_of_up, number_of_down
numerical_df["number_of_no"] = 0
numerical_df["number_of_steady"] = 0
numerical_df["number_of_up"] = 0
numerical_df["number_of_down"] = 0

C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1141929488.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_df["number_of_no"] = 0
C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1141929488.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_df["number_of_steady"] = 0
C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1141929488.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [35]:
for drug in drugs:
    numerical_df["number_of_no"] += (categorical_df[drug] == 'No').astype(int)
    numerical_df["number_of_steady"] += (categorical_df[drug] == 'Steady').astype(int)
    numerical_df["number_of_up"] += (categorical_df[drug] == 'Up').astype(int)
    numerical_df["number_of_down"] += (categorical_df[drug] == 'Down').astype(int)

C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1055870108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_df["number_of_no"] += (categorical_df[drug] == 'No').astype(int)
C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1055870108.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_df["number_of_steady"] += (categorical_df[drug] == 'Steady').astype(int)
C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1055870108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [36]:
# categorical_df.head(10)

In [37]:
# drop those drugs columns
categorical_df = categorical_df.drop(columns=drugs,axis=1)

## Feature Engineering for Patient_Id

In [38]:
lst = categorical_df['patient_id'].to_list()
pat = {}
for i in lst:
    if i in pat:
        pat[i] += 1
    else:
        pat[i] = 1

In [39]:
on = [i for i in pat if pat[i] == 1]
tw = [i for i in pat if pat[i] == 2]
mrth = [i for i in pat if pat[i] > 2]

In [40]:
def calc_bin(id):
    if id in on:
        return 1
    elif id in tw:
        return 2
    elif id in mrth:
        return 3
    else:
        return 0

In [41]:
categorical_df['new_patient_id'] = categorical_df['patient_id'].apply(lambda x: calc_bin(x)).astype(int)

In [42]:
categorical_df['new_patient_id'].value_counts()

new_patient_id
1    43654
2    13982
3    13600
Name: count, dtype: int64

In [43]:
categorical_df = categorical_df.drop(columns='patient_id',axis=1)

In [44]:
categorical_df.head()

,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,change,diabetesMed,readmission_id,new_patient_id
0,1,[60-70),1,2,4,410,428,414,None,None,1,1,2,1
1,1,[70-80),1,1,1,414,424,428,None,None,0,1,1,2
2,0,[70-80),0,2,1,518,514,997,None,None,1,1,1,3
3,0,[50-60),1,1,7,592,591,250,None,None,0,1,2,1
4,0,[80-90),1,1,7,250.6,713,707,None,None,1,1,2,1


## Training

In [45]:
label_encoder=LabelEncoder()
categorical_df['diag_1']=label_encoder.fit_transform(categorical_df['diag_1'])
categorical_df['diag_2']=label_encoder.fit_transform(categorical_df['diag_2'])
categorical_df['diag_3']=label_encoder.fit_transform(categorical_df['diag_3'])
categorical_df['age']=label_encoder.fit_transform(categorical_df['age'])
categorical_df['A1Cresult']=label_encoder.fit_transform(categorical_df['A1Cresult'])
categorical_df['max_glu_serum']=label_encoder.fit_transform(categorical_df['max_glu_serum'])

In [46]:
X=categorical_df.drop('readmission_id',axis=1)
y=categorical_df['readmission_id']
X=pd.concat([X,numerical_df],axis=1)
print(X.shape)

(71236, 25)


In [269]:
# # use grid to search to find the best parameters for xgboost
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import RandomizedSearchCV

# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [4, 5, 6, 7]
#         }
# # Initialize XGB and GridSearch
# xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
# grid_search = GridSearchCV(xgb_model, params, scoring='accuracy', cv=5, n_jobs=-1, verbose=2)

In [270]:
# use grid search to find the best parameters for random forest
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [5, 7, 10, 12],
    'min_samples_split': [5, 10, 12],
    'min_samples_leaf': [1, 2, 4, 6]
}

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42,stratify=y)

In [50]:
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71236 entries, 0 to 71235
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   gender                    71236 non-null  int64
 1   age                       71236 non-null  int32
 2   admission_type_id         71236 non-null  int64
 3   discharge_disposition_id  71236 non-null  int64
 4   admission_source_id       71236 non-null  int64
 5   diag_1                    71236 non-null  int32
 6   diag_2                    71236 non-null  int32
 7   diag_3                    71236 non-null  int32
 8   max_glu_serum             71236 non-null  int32
 9   A1Cresult                 71236 non-null  int32
 10  change                    71236 non-null  int64
 11  diabetesMed               71236 non-null  int64
 12  readmission_id            71236 non-null  int64
 13  new_patient_id            71236 non-null  int32
dtypes: int32(7), int64(7)
memory usage: 5.

In [272]:
# grid_search.fit(X_train, y_train)
# print(grid_search.best_params_)
# rm = grid_search.best_estimator_

In [55]:
import xgboost as xgb
rm = xgb.XGBClassifier(
    objective="multi:softprob" 
)
# rm=RandomForestClassifier()
# rm = RandomForestClassifier(
#     max_leaf_nodes=1600,
#     max_depth=300
# )

In [56]:
rm.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [57]:
y_pred = rm.predict(X_test)
pd.crosstab(pd.Series(y_test, name = 'Actual'), pd.Series(y_pred, name = 'Predict'), margins = True)

Predict,0,1,2,All
Actual,,,,
0,4,108,186,298
1,14,328,661,1003
2,23,499,1016,1538
All,41,935,1863,2839


In [58]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
print("Accuracy is {0:.2f}".format(accuracy_score(y_test, y_pred)))
print("micro: {:.2f}".format(metrics.precision_score(y_test, y_pred, average='micro')))
print("macro: {:.2f} ".format( metrics.precision_score(y_test, y_pred, average='macro')))
print("weighted: {:.2f} ".format( metrics.precision_score(y_test, y_pred, average='weighted'))) 
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy is 0.68
micro: 0.68
macro: 0.58 
weighted: 0.65 
              precision    recall  f1-score   support

           0       0.41      0.05      0.08      1590
           1       0.61      0.57      0.59      4977
           2       0.73      0.89      0.80      7681

    accuracy                           0.68     14248
   macro avg       0.58      0.50      0.49     14248
weighted avg       0.65      0.68      0.65     14248

[[  75 1004  511]
 [  83 2843 2051]
 [  27  844 6810]]


## Test data

In [60]:
test=pd.read_csv("data/test.csv")

In [61]:
test['A1Cresult'] = test['A1Cresult'].replace(np.nan, 'None')
test['max_glu_serum'] = test['max_glu_serum'].replace(np.nan, 'None')
test.head()

,enc_id,patient_id,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,86305392,23846112,Caucasian,Male,[40-50),NaN,5,5,17,12,...,No,No,Steady,No,No,No,No,No,No,Yes
1,394919696,90620001,Caucasian,Male,[80-90),NaN,2,3,7,3,...,No,No,Down,No,No,No,No,No,Ch,Yes
2,164917446,25342173,AfricanAmerican,Female,[60-70),NaN,3,1,1,8,...,No,No,Down,No,No,No,No,No,Ch,Yes
3,178319040,23481783,AfricanAmerican,Female,[70-80),NaN,1,3,7,10,...,No,No,Up,No,No,No,No,No,Ch,Yes
4,253585416,86500791,Caucasian,Male,[70-80),NaN,1,6,7,5,...,No,No,No,No,No,No,No,No,No,No


In [62]:
# for i in range(0,10):
#     test['age'] = test['age'].replace('['+str(10*i)+'-'+str(10*(i+1))+')', i)
# test['age'].value_counts()

In [63]:
# categorical_test_df=test.drop(['enc_id','weight','race','medical_specialty',
#        'time_in_hospital', 'payer_code',
#        'num_lab_procedures', 'num_procedures', 'num_medications',
#        'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses','A1Cresult','max_glu_serum'],axis=1)
categorical_test_df=test.drop(['enc_id','weight','race','medical_specialty',
       'time_in_hospital', 'payer_code',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses'],axis=1)
print(categorical_test_df.shape)

(30530, 36)


In [64]:
numerical_test_df = test[['num_medications','time_in_hospital','num_procedures','num_lab_procedures','number_diagnoses','number_emergency','number_inpatient',
'number_outpatient']]

## Feature Engineering for Drug columns

In [65]:
numerical_test_df["number_of_no"] = 0
numerical_test_df["number_of_steady"] = 0
numerical_test_df["number_of_up"] = 0
numerical_test_df["number_of_down"] = 0

C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1767665680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_test_df["number_of_no"] = 0
C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1767665680.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_test_df["number_of_steady"] = 0
C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\1767665680.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [66]:
for drug in drugs:
    numerical_test_df["number_of_no"] += (categorical_test_df[drug] == 'No').astype(int)
    numerical_test_df["number_of_steady"] += (categorical_test_df[drug] == 'Steady').astype(int)
    numerical_test_df["number_of_up"] += (categorical_test_df[drug] == 'Up').astype(int)
    numerical_test_df["number_of_down"] += (categorical_test_df[drug] == 'Down').astype(int)

C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\635781213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_test_df["number_of_no"] += (categorical_test_df[drug] == 'No').astype(int)
C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\635781213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_test_df["number_of_steady"] += (categorical_test_df[drug] == 'Steady').astype(int)
C:\Users\rlsha\AppData\Local\Temp\ipykernel_19692\635781213.py:4: SettingWithCopyWarning: 
A value is trying t

In [67]:
# drop those drugs columns
categorical_test_df = categorical_test_df.drop(columns=drugs,axis=1)

In [68]:
label_encoder=LabelEncoder()
categorical_test_df['diag_1']=label_encoder.fit_transform(categorical_test_df['diag_1'])
categorical_test_df['diag_2']=label_encoder.fit_transform(categorical_test_df['diag_2'])
categorical_test_df['diag_3']=label_encoder.fit_transform(categorical_test_df['diag_3'])
categorical_test_df['age']=label_encoder.fit_transform(categorical_test_df['age'])
categorical_test_df['A1Cresult']=label_encoder.fit_transform(categorical_test_df['A1Cresult'])
categorical_test_df['max_glu_serum']=label_encoder.fit_transform(categorical_test_df['max_glu_serum'])

In [69]:
categorical_test_df['change'] = categorical_test_df['change'].replace('Ch', 1)
categorical_test_df['change'] = categorical_test_df['change'].replace('No', 0)
categorical_test_df['gender'] = categorical_test_df['gender'].replace('Male', 1)
categorical_test_df['gender'] = categorical_test_df['gender'].replace('Female', 0)
categorical_test_df['gender'] = categorical_test_df['gender'].replace('Unknown/Invalid', 0)
categorical_test_df['diabetesMed'] = categorical_test_df['diabetesMed'].replace('Yes', 1)
categorical_test_df['diabetesMed'] = categorical_test_df['diabetesMed'].replace('No', 0)
# test_df["drug_change_sum"]=0
# for col in drugs:
#     test_df[col] = test_df[col].replace('No', 0)
#     test_df[col] = test_df[col].replace('Steady', 0)
#     test_df[col] = test_df[col].replace('Up', 1)
#     test_df[col] = test_df[col].replace('Down', 1)
# for col in drugs:
#     test_df["drug_change_sum"]+=test_df[col]
# test_df=test_df.drop(columns=drugs,axis=1)
# print(test_df.shape)

In [70]:
print(categorical_test_df.isna().sum())

patient_id                  0
gender                      0
age                         0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
diag_1                      0
diag_2                      0
diag_3                      0
max_glu_serum               0
A1Cresult                   0
change                      0
diabetesMed                 0
dtype: int64


In [71]:
lst = categorical_test_df['patient_id'].to_list()
pat = {}
for i in lst:
    if i in pat:
        pat[i] += 1
    else:
        pat[i] = 1
on = [i for i in pat if pat[i] == 1]
tw = [i for i in pat if pat[i] == 2]
mrth = [i for i in pat if pat[i] > 2]
def calc_bin(id):
    if id in on:
        return 1
    elif id in tw:
        return 2
    elif id in mrth:
        return 3
    else:
        return 0
categorical_test_df['new_patient_id'] = categorical_test_df['patient_id'].apply(lambda x: calc_bin(x)).astype(int)
categorical_test_df = categorical_test_df.drop(columns='patient_id',axis=1)

In [72]:
categorical_test_df['admission_type_id'] = categorical_test_df['admission_type_id'].replace(2,1)
categorical_test_df['admission_type_id'] = categorical_test_df['admission_type_id'].replace(7,1)
categorical_test_df['admission_type_id'] = categorical_test_df['admission_type_id'].replace(6,5)
categorical_test_df['admission_type_id'] = categorical_test_df['admission_type_id'].replace(8,5)
print(categorical_test_df['admission_type_id'].value_counts())
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(6,1)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(8,1)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(9,1)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(13,1)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(3,2)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(4,2)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(5,2)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(14,2)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(22,2)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(23,2)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(24,2)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(12,10)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(15,10)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(16,10)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(17,10)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(25,18)
categorical_test_df['discharge_disposition_id'] = categorical_test_df['discharge_disposition_id'].replace(26,18)

categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(2,1)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(3,1)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(5,4)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(6,4)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(10,4)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(22,4)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(25,4)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(15,9)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(17,9)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(20,9)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(21,9)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(13,11)
categorical_test_df['admission_source_id'] = categorical_test_df['admission_source_id'].replace(14,11)

admission_type_id
1    21674
3     5681
5     3173
4        2
Name: count, dtype: int64


In [73]:
test_df=pd.concat([categorical_test_df,numerical_test_df],axis=1)
test_df.fillna(0)
test_df.head()

,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,...,num_procedures,num_lab_procedures,number_diagnoses,number_emergency,number_inpatient,number_outpatient,number_of_no,number_of_steady,number_of_up,number_of_down
0,1,4,5,2,9,332,528,391,3,2,...,3,25,9,0,4,0,22,1,0,0
1,1,8,1,2,7,235,265,217,2,1,...,0,59,9,1,0,0,22,0,0,1
2,0,6,3,1,1,235,329,210,2,2,...,0,47,9,0,0,0,22,0,0,1
3,0,7,1,2,7,219,530,120,2,2,...,6,46,9,2,7,0,21,1,1,0
4,1,7,1,1,7,235,223,217,2,2,...,1,64,9,0,1,0,23,0,0,0


In [74]:
test_pred = rm.predict(test_df)

In [75]:
test_df['readmission_id']=test_pred
test_df['enc_id']=test['enc_id']
print(test_df.shape)
cvfile = pd.concat((test_df["enc_id"], test_df["readmission_id"]), axis=1)
cvfile.head()
cvfile.to_csv('finale.csv', index=False)

(30530, 27)
